# Xena Differential Gene Expression Analysis Pipeline

This pipeline enables you to analyze and visualize Xena's gene expression datasets with an array of downstream analysis and visualization tools. The pipeline includes: PCA/t-SNE analysis, total gene expression analysis, differential gene expression analysis, pathway enrichment analysis, and L1000 small molecule search. The pipeline is adapted from Maayan lab bulk RNA-seq analysis [appyter here](https://appyters.maayanlab.cloud/#/Bulk_RNA_seq)

In [ ]:
#%%appyter init
from appyter import magic
magic.init(lambda _=globals: _())

In [ ]:
# Basic libraries
import pandas as pd
import os
import urllib3
import requests, json
import sys
import geode
import random
from time import sleep
import time
import numpy as np
import warnings
import base64  
from pandas.api.types import CategoricalDtype
from io import BytesIO

# Visualization
import plotly
from plotly import tools
import plotly.express as px
import plotly.graph_objs as go
plotly.offline.init_notebook_mode() # To embed plots in the output cell of the notebook

import matplotlib.pyplot as plt; plt.rcdefaults()
from matplotlib import rcParams
from matplotlib.lines import Line2D
from matplotlib_venn import venn2, venn3
%matplotlib inline

import IPython
from IPython.display import HTML, display, Markdown, IFrame

import chart_studio
import chart_studio.plotly as py


# Data analysis
from itertools import combinations
import scipy.spatial.distance as dist
import scipy.stats as ss
from sklearn.decomposition import PCA
from sklearn.preprocessing import quantile_transform

from rpy2 import robjects
from rpy2.robjects import r, pandas2ri

# External Code
from utils import *

In [ ]:
%%appyter hide_code_exec
{% do SectionField(
    name='Subgroup_Section',
    title='Set Up the Two Subgroups for Differential Gene Expression Analysis',
    subtitle='',
    img='analysis.png'
) %}

{% do SectionField(
    name='Visualization_Section',
    title='Advanced Visualization Parameters',
    subtitle='',
    img='analysis.png',
    collapse= "hide"
) %}

{% do SectionField(
    name='DEG_Section',
    title='Advanced Differentially Exprssed Gene Analysis Parameters',
    subtitle='',
    img='analysis.png',
    collapse= "hide"
) %}

In [ ]:
%%appyter code_exec
{% set DE_params = PMSGStringField(
    name='DE_params', 
    label='Differential expression parameters', 
    default='',
    section='Subgroup_Section') 
%}

In [ ]:
%%appyter code_exec
{% set case_group = CategoryMultiChoiceField(
    name='category_case',
    urlhashkey='category',
    label='Subgroup 1',
    description='Select category to be in the 1st subgroup',
    section='Subgroup_Section')
%}

{% set control_group = CategoryMultiChoiceField(
    name='category_control',
    urlhashkey='category',
    label='Subgroup 2',
    description='Select category to be in the 2nd subgroup',
    section='Subgroup_Section')
%}

In [ ]:
%%appyter code_exec
{% set interactive_plot = BoolField(
    name='interactive_plot', 
    label='Interactive plots?', 
    default='true', 
    description='Check if User wants interactive plots', 
    section='Visualization_Section')
%}

{% set visualization_method = ChoiceField(
    name='visualization_method', 
    label='Visualization Methods', 
    choices = {'PCA': 'PCA', 't-SNE': 't-SNE'},
    default='PCA', 
    description='Select a visualization method', 
    section='Visualization_Section')
%}

{% set nr_genes = IntField(
    name='nr_genes', 
    label='Genes for Dimension Reduction', 
    min=0, 
    max=30000, 
    default=2500, 
    description='The maximum number of genes for dimension reduction', 
    section='Visualization_Section')
%}


In [ ]:
%%appyter code_exec
{% set filter_genes = BoolField(
    name='filter_genes', 
    label='Filter genes?', 
    default='false',
    description='Check if User wants to filter genes with lowest variances', 
    section='Visualization_Section',
) 
%}

{% set low_expression_threshold = FloatField(
    name='low_expression_threshold', 
    label='Low expression threshold', 
    default=0.3, 
    description='Threshold to filter out low expression genes. The value should vary based on the user dataset.', 
    section='Visualization_Section'
)
%}

{% set logCPM_normalization = BoolField(
    name='logCPM_normalization', 
    label='logCPM normalization?', 
    default='false', 
    description='Check if User wants the dataset to be logCPM-transformed', 
    section='Visualization_Section')
%}

{% set log_normalization = LogNormalizationField(
    name='log_normalization',
    label='log normalization?',
    urlhashkey = 'expression_unit',
    default='false', 
    description='Check if User wants the dataset to be log-transformed', 
    section='Visualization_Section')
%}

{% set z_normalization = BoolField(
    name='z_normalization', 
    urlhashkey = 'expression_unit',
    label='Z normalization?', 
    default='true', 
    description='Check if User wants the dataset to be normalized with Z-normalized method', 
    section='Visualization_Section')
%}

{% set q_normalization = BoolField(
    name='q_normalization', 
    label='Quantile normalization?', 
    default='false', 
    description='Check if User wants the dataset to be normalized with Quantile normalization method', 
    section='Visualization_Section')
%}

In [ ]:
%%appyter code_exec
{% set diff_gex_method = DiffgexChoiceField(
    name='DiffgexChoiceField',
    label='Differential expression analysis method',
    urlhashkey = 'expression_unit',
    choices={'limma': 'limma','limma_voom': 'limma_voom', 'characteristic direction': 'characteristic_direction', 'edgeR': 'edgeR', 'DESeq2': 'DESeq2'},
    default='limma', 
    description='Set a method to get differentially expressed genes', 
    section='DEG_Section')
%}

{% set diff_gex_plot_method = ChoiceField(
    name='diff_gex_plot_method',
    label='Differential expression analysis plotting method',
    choices={'Volcano plot': 'volcano','MA plot': 'MA_plot'},
    default='Volcano plot', 
    description='Set a plot method to see differentially expressed genes. Available for limma/edgeR/DESeq2.', 
    section='DEG_Section')
%}

{% set pvalue_threshold = FloatField(
    name='pvalue_threshold', 
    label='P-value threshold', 
    min=0, 
    max=1, 
    default=0.05, 
    description='Threshold to highlight significantly differentially expressed genes.', 
    section='DEG_Section')
%}
{% set logfc_threshold = FloatField(
    name='logfc_threshold', 
    label='logFC threshold',
    min=0,
    max=1000,
    default=1.5, 
    description='Threshold to highlight diffentially expressed genes.', 
    section='DEG_Section')
%}

{% set gene_topk = IntField(
    name='gene_topk', 
    label='Maximum genes for Enrichr', 
    min=0, 
    max=1000, 
    default=500, 
    description='The maximum number of genes discovered by the Characteristic Direction method', 
    section='DEG_Section')
%}


{% set enrichr_libraries = MultiChoiceField(
    name='enrichr_libraries',
    label='Enrichr Libraries',
    description='Enrichr libraries to be visualized. Select one or two libraries',
    choices=['Gene Ontology',
            'Pathway',
            'Kinase',
            'Transcription Factor',
            'miRNA'],
    default=['Gene Ontology', 'Pathway'],
    section='DEG_Section'
    )
%}


{% set nr_genesets = IntField(
    name='nr_genesets', 
    label='Top ranked gene sets', 
    min=0, 
    max=20, 
    default=15, 
    description='The number of result gene sets', 
    section='DEG_Section')
%}

{% set small_molecule_method = ChoiceField(
    name='small_molecule_method',
    label='Small molecule analysis method',
    choices={'L1000CDS2': 'L1000CDS2','L1000FWD': 'L1000FWD'},
    default='L1000FWD', 
    description='Set a small molecule analysis method', 
    section='DEG_Section')
%}

{% set l1000_topk = IntField(
    name='l1000_topk', 
    label='Genes for L1000CDS2 or L1000FWD', 
    min=0, 
    max=1000, 
    default=500, 
    description='The number of genes to L1000CDS2 or L1000FWD', 
    section='DEG_Section')
%}

{% set nr_drugs = IntField(
    name='nr_drugs', 
    label='Top ranked drugs from L1000CDS2 or L1000FWD', 
    min=0, 
    max=20, 
    default=7, 
    description='The number of result drugs', 
    section='DEG_Section')
%}


In [ ]:
%%appyter code_exec
# Parse Xena Browser post data
DE_params = """{{DE_params.value}}"""
J = json.loads(DE_params)

expr_dataset_name = J['preferredExpression']['name']
xena_host = J['preferredExpression']['host']
probemap_name = J['preferredExpression']['probemap']
expr_data_filename = xenaFileDownloadLink(xena_host, expr_dataset_name)
probemap_filename = xenaFileDownloadLink(xena_host, probemap_name)
expression_unit = J['preferredExpression']['unit']
samples = J["samples"]
values = J["data"]["req"]["values"][0]
codes = J["data"]["codes"]
assert(len(samples) == len(values))

In [ ]:
%%appyter code_exec
case_group = {{case_group.value}}
control_group = {{control_group.value}}

interactive_plot = {{interactive_plot.value}}
filter_genes = {{filter_genes.value}}
low_expression_threshold ={{low_expression_threshold.value}}
nr_genes = {{nr_genes.value}}
logCPM_normalization= {{logCPM_normalization}}
log_normalization= {{log_normalization}}
z_normalization= {{z_normalization}}
q_normalization= {{q_normalization}}

diff_gex_method = "{{diff_gex_method.value}}"
diff_gex_plot_method = "{{diff_gex_plot_method.value}}"
pvalue_threshold = {{pvalue_threshold.value}}
logfc_threshold = {{logfc_threshold.value}}
gene_topk = {{gene_topk.value}}
enrichr_libraries = {{enrichr_libraries.value}}
nr_genesets = {{nr_genesets.value}}
enrichr_success = True

small_molecule_method = "{{small_molecule_method.value}}"
l1000_topk = {{l1000_topk.value}}
nr_drugs = {{nr_drugs.value}}

In [ ]:
warnings.filterwarnings('ignore')
random.seed(0)
pandas2ri.activate()
if interactive_plot == True:
    plot_type='interactive'
else:
    plot_type='static'
chart_studio.tools.set_credentials_file(username='mjjeon', api_key='v0rpMa6lhST28Sq7XqtM')
results = {}
table_counter = 1
figure_counter = 1

# Load datasets

In [ ]:
# Downloading expr data file
try:
    expr_data_filename = find_remote_file(expr_data_filename)
except:
    print(f"Error! Gene expression file not found at '{expr_data_filename}'")

expr_buffer = get_Remote_File_Buffer(expr_data_filename)
if expr_data_filename.endswith(".gz"):
    expr_df = pd.read_csv(expr_buffer, index_col=0, sep="\t", compression="gzip").sort_index()
else:
    expr_df = pd.read_csv(expr_buffer, index_col=0, sep="\t", compression="infer").sort_index()

meta_df = build_meta_df(samples, values, codes)

# display expr table
table_counter = display_object(table_counter, "Gene expression data. The table displays the first 5 rows of the input gene expression dataset. Rows represent genes, columns represent samples, and values show the gene expression level.", expr_df.head(), istable=True)

# display meta data table
table_counter = display_object(table_counter, "Metadata. The table displays the metadata associated with the samples in the gene expression dataset. Rows represent samples, columns represent metadata categories and subgroups.", meta_df.head(), istable=True)


# Build Subgroup 1 vs. Subgroup 2

In [ ]:
meta_id_column_name, meta_class_column_name = meta_df.columns
meta_subgroup_column_name = "subgroup"

# build case vs. control subgroups
try:
    check_subgroups (control_group, case_group)
except Exception as e:
    print(f"Error! '{e}'")
case_subgroup_name = '+'.join(case_group)
control_subgroup_name = '+'.join(control_group)
meta_df[meta_subgroup_column_name] = np.where(np.isin(meta_df[meta_class_column_name], case_group), case_subgroup_name, np.where(np.isin(meta_df[meta_class_column_name], control_group), control_subgroup_name,''))
subgroups = [control_subgroup_name, case_subgroup_name] # CONTROL first, CASE second

table_counter = display_object(table_counter, "Sample size for each category. The table displays the number of samples in each category.", meta_df.groupby(meta_class_column_name)[meta_class_column_name].count().to_frame(), istable=True)
table_counter = display_object(table_counter, "Sample size for each subgroup. The table displays the number of samples in each subgroup.", meta_df.groupby(meta_subgroup_column_name)[meta_subgroup_column_name].count().to_frame(), istable=True)

In [ ]:
# Match samples between the metadata (subgroup column) and the expression datasets
try:
    check_df(meta_df, meta_id_column_name)
except:
    print(f"Error! Column '{meta_id_column_name}' is not in metadata")
try:
    check_df(meta_df, meta_class_column_name)
except:
    print(f"Error! Column '{meta_class_column_name}' is not in metadata")
meta_df = meta_df[meta_df[meta_subgroup_column_name]!='']
meta_df = meta_df[meta_df[meta_id_column_name].isin(expr_df.columns)]
expr_df = expr_df.loc[:,meta_df[meta_id_column_name]]
expr_df = expr_df.groupby(expr_df.index).sum() # add all values from the same gene

try:
    assert(meta_df.shape[0]==expr_df.shape[1])
except:
    print("Error! Input files are in a wrong format. \
    Please check if the index of the expression data are genes and the columns are sample IDs. \
    Sample IDs in the expression data and the metadata should be matched")

# convert non-HUGO probe_level expr_df to hugo-gene-level expr_df
to_hugo = False
if probemap_filename != 'None':
    probemap = probemap_filename.split('/')[-1]
    if probemap not in probemap_level or probemap_level[probemap] != "HUGO":
        try:
            probemap_filename = find_remote_file(probemap_filename)
        except:
            print(f"Error! Probemap file not found at '{probemap_filename}'")
    
        probemap_buffer = get_Remote_File_Buffer(probemap_filename)
        if probemap_filename.endswith(".gz"):
            probemap_df = pd.read_csv(probemap_buffer, index_col=0, sep="\t", compression="gzip")
        else:
            probemap_df = pd.read_csv(probemap_buffer, index_col=0, sep="\t", compression="infer")
        if check_probe_level(probemap_df) != "HUGO":
            # convert expr_df to hugo_gene_level expr_df
            expr_df = convert_to_hugo(expr_df, probemap_df)
            to_hugo = True

dataset = dict()
current_dataset = 'rawdata'
dataset[current_dataset] = expr_df
dataset['dataset_metadata'] = meta_df


In [ ]:
%%appyter markdown
{% if filter_genes.value == True %}
Filter out low expressed genes 
{% endif %}

In [ ]:
%%appyter code_exec
{% if filter_genes.value == True %}
## Filter out non-expressed genes
expr_df = expr_df.loc[expr_df.sum(axis=1) > 0, :]

## Filter out lowly expressed genes
mask_low_vals = (expr_df > low_expression_threshold).sum(axis=1) > 2
expr_df = expr_df.loc[mask_low_vals, :]
current_dataset += '+filter_genes'
dataset[current_dataset] = expr_df
{% endif %}

In [ ]:
# display expression data table
if (to_hugo):
    table_counter = display_object(table_counter, "Gene expression data. The table displays the first 5 rows of the quantified gene expression dataset after converting input data to using hugo gene names. Rows represent genes, columns represent samples, and values show the gene expression level.", dataset[current_dataset].head(), istable=True)
    display(create_download_link(dataset[current_dataset], filename="gene_level_expr.csv"))

In [ ]:
%%appyter markdown
{% if logCPM_normalization.value == True or log_normalization.value == True or z_normalization.value == True or q_normalization.value == True %}
Normalization methods ({% if logCPM_normalization.value %}count per million (CPM), {% endif %} {% if log_normalization.value %} log transformation, {% endif %} {% if z_normalization.value %} Z normalization, {% endif %} {% if q_normalization.value %}quantile normalization {% endif %}) will be applied to convert raw read counts into informative measures of gene expression and remove factors that affect the analysis.
{% endif %}

In [ ]:
%%appyter code_exec
dataset, normalization = normalize(dataset, current_dataset, {{logCPM_normalization}}, {{log_normalization}}, {{z_normalization}}, {{q_normalization}})

{% if logCPM_normalization.value == True or log_normalization.value == True or z_normalization.value == True or q_normalization.value == True %}
table_counter = display_object(table_counter, "Normalized data. The table displays the expression values after normalization.", dataset[normalization].head(), istable=True)
display(create_download_link(dataset[normalization], filename="normalized_data.csv"))
{% endif %}

# Visualize Samples

In [ ]:
%%appyter markdown
{% if visualization_method.value == "PCA" %}
Principal Component Analysis (PCA) (Clark et al. 2011) is a statistical technique used to identify global patterns in high-dimensional datasets. It is commonly used to explore the similarity of biological samples in gene expression datasets. To achieve this, gene expression values are transformed into Principal Components (PCs), a set of linearly uncorrelated features which represent the most relevant sources of variance in the data, and subsequently visualized using a scatter plot.
{% endif %}

In [ ]:
%%appyter code_exec
{% if visualization_method.value == "PCA" %}
method = "PCA"
{% endif %}
{% if visualization_method.value == "t-SNE" %}
method = "t-SNE"
{% endif %}
# Run analysis
results[method] = run_dimension_reduction(dataset=dataset, meta_id_column_name=meta_id_column_name, method=method,\
                         nr_genes=nr_genes, normalization=normalization, plot_type=plot_type)
# Display results
figure_counter = plot_samples(results[method], meta_id_column_name=meta_id_column_name, meta_class_column_name=meta_class_column_name, counter=figure_counter)


# Total gene expression analysis

Total gene expression analysis calculates and displays the total gene expression for each sample in the gene expression dataset, facilitating the identification of outlying samples and the assessment of the overall quality of the data.

In [ ]:
meta_df['sum'] = expr_df.sum().tolist()
fig = px.bar(meta_df.sort_values(by='sum'), x=meta_id_column_name, y='sum')

if plot_type=='interactive':
        plotly.offline.iplot(fig)
else:
    py.image.ishow(fig)
figure_counter = display_object(figure_counter, "The total gene expression for each sample. The figure contains an interactive bar chart which displays the total gene expression (sum) mapped to each sample in the dataset. Additional information for each sample is available by hovering over the bars.", istable=False)

# Differential Gene Expression 

Gene expression signatures are alterations in the patterns of gene expression that occur as a result of cellular perturbations such as drug treatments, gene knock-downs or diseases. They can be quantified using differential gene expression (DGE) methods (Ritchie et al. 2015, Clark et al. 2014), which compare gene expression between two groups of samples to identify genes whose expression is significantly altered in the perturbation. 

In [ ]:
signatures = get_signatures(subgroups, dataset, normalization, diff_gex_method, meta_subgroup_column_name, meta_id_column_name, filter_genes)

for label, signature in signatures.items():
    table_counter = display_object(table_counter, "Differentially expressed genes between {} using {}. The figure displays a browsable table containing the gene expression signature generated from a differential gene expression analysis. Every row of the table represents a gene; the columns display the estimated measures of differential expression.".format(label, diff_gex_method), signature, istable=True)
    display(create_download_link(signature, filename="DEG_results_{}.csv".format(label)))

In [ ]:
%%appyter code_exec
{% if diff_gex_method.value == "limma" or diff_gex_method.value == "limma_voom" or diff_gex_method.value == "edgeR" or diff_gex_method.value == "DESeq2"%}
    
{% if diff_gex_plot_method.value == "volcano" %}
results['volcano_plot'] = {}
# Loop through signatures
for label, signature in signatures.items():
    results['volcano_plot'][label] = run_volcano(signature, label, dataset, pvalue_threshold, logfc_threshold, plot_type)
    plot_volcano(results['volcano_plot'][label])
    figure_counter = display_object(figure_counter, "Volcano plot for {}. The figure contains an interactive scatter plot which displays the log2-fold changes and statistical significance of each gene calculated by performing a differential gene expression analysis. Genes with logFC > {} and p-value < {} in red and genes with logFC < -{} and p-value < {} in blue. Additional information for each gene is available by hovering over it.".format(label, logfc_threshold, pvalue_threshold, logfc_threshold, pvalue_threshold), istable=False)

{% elif diff_gex_plot_method.value == "MA_plot" %}
# Initialize results
results['ma_plot'] = {}

# Loop through signatures
for label, signature in signatures.items():
    # Run analysis
    results['ma_plot'][label] = run_maplot(signature=signature, signature_label=label, pvalue_threshold=pvalue_threshold, logfc_threshold=logfc_threshold, plot_type=plot_type)
    # Display results
    plot_maplot(results['ma_plot'][label])
    figure_counter = display_object(figure_counter, "MA plot for {}. The figure contains an interactive scatter plot which displays the average expression and statistical significance of each gene calculated by performing differential gene expression analysis. Genes with logFC > {} and p-value < {} in red and genes with logFC < -{} and p-value < {} in blue. Additional information for each gene is available by hovering over it.".format(label, logfc_threshold, pvalue_threshold, logfc_threshold, pvalue_threshold), istable=False)

{% endif %}
{% endif %}

# Enrichment Analysis using Enrichr

Enrichment analysis is a statistical procedure used to identify biological terms which are over-represented in a given gene set. These include signaling pathways, molecular functions, diseases, and a wide variety of other biological terms obtained by integrating prior knowledge of gene function from multiple resources. Enrichr (Kuleshov et al. 2016) is a web-based application which allows to perform enrichment analysis using a large collection of gene-set libraries and various interactive approaches to display enrichment results.

In [ ]:
# Loop through signatures
results = {}
results['enrichr']= {}
enrichr_success = True
if diff_gex_method == "characteristic_direction":
    fc_colname = "CD-coefficient"
    sort_genes_by = "CD-coefficient"
    ascending = False
elif diff_gex_method == "limma" or diff_gex_method == "limma_voom":
    fc_colname = "logFC"
    sort_genes_by = "t"
    ascending = False
elif diff_gex_method == "edgeR":
    fc_colname = "logFC"
    sort_genes_by = "PValue"
    ascending = True
elif diff_gex_method == "DESeq2":
    fc_colname = "log2FoldChange"
    sort_genes_by = "padj"
    ascending = True
for label, signature in signatures.items():
    # Run analysis
    try:
        results['enrichr'][label] = run_enrichr(signature=signature, signature_label=label, fc_colname=fc_colname,geneset_size=gene_topk, sort_genes_by = sort_genes_by,ascending=ascending)
        display(Markdown("*Enrichment Analysis Result: {} ({}-up)*".format(label, case_subgroup_name)))
        display_link("https://amp.pharm.mssm.edu/Enrichr/enrich?dataset={}".format(results['enrichr'][label]["upregulated"]["shortId"]))
        display(Markdown("*Enrichment Analysis Result: {} ({}-down)*".format(label, case_subgroup_name)))
        display_link("https://amp.pharm.mssm.edu/Enrichr/enrich?dataset={}".format(results['enrichr'][label]["downregulated"]["shortId"]))
        table_counter = display_object(table_counter, "The table displays links to Enrichr containing the results of enrichment analyses generated by analyzing the up-regulated and down-regulated genes from a differential expression analysis. By clicking on these links, users can interactively explore and download the enrichment results from the Enrichr website.")
    except Exception as e:
        print(f"Enrichr Error! '{e}'")
        print('Skip all pathway enrichment analysis')
        enrichr_success = False

In [ ]:
# GO Enrichment Analysis
if "Gene Ontology" in enrichr_libraries and enrichr_success:    
    results['go_enrichment'] = {}

    for label, signature in signatures.items():
        # Run analysis
        results['go_enrichment'][label] = get_enrichr_results_by_library(results['enrichr'][label], label, library_type='go', version='2018')

    for label, signature in signatures.items():
        # Create dataframe
        enrichment_results = results['go_enrichment'][label]
        enrichment_dataframe = pd.concat([enrichment_results['upregulated'], enrichment_results['downregulated']])

        # Plot barcharts
        libraries = enrichment_dataframe['gene_set_library'].unique()   
        if (len(libraries)):
            display(Markdown("## GO Enrichment Analysis"))
            display(Markdown("Gene Ontology (GO) (Ashburner et al. 2000) is a major bioinformatics initiative aimed at unifying the representation of gene attributes across all species. It contains a large collection of experimentally validated and predicted associations between genes and biological terms. This information can be leveraged by Enrichr to identify the biological processes, molecular functions and cellular components which are over-represented in the up-regulated and down-regulated genes identified by comparing two groups of samples"))
            
            for gene_set_library in libraries:
                plot_library_barchart(enrichment_results, gene_set_library, enrichment_results['signature_label'], enrichment_results['sort_results_by'], nr_genesets=nr_genesets, plot_type=plot_type) # 10 300
            
            figure_counter = display_object(figure_counter, "Enrichment Analysis Results for {} in Gene Onotology. The figure contains interactive bar charts displaying the results of the Gene Ontology enrichment analysis generated using Enrichr. The x axis indicates the -log10(P-value) for each term. Significant terms are highlighted in bold. Additional information about enrichment results is available by hovering over each bar.".format(label), istable=False)

In [ ]:
# Pathway Enrichment Analysis
if "Pathway" in enrichr_libraries and enrichr_success:
    results['pathway_enrichment'] = {}
    for label, signature in signatures.items():
        # Run analysis
        results['pathway_enrichment'][label] = get_enrichr_results_by_library(results['enrichr'][label], label, library_type='pathway')

    for label, signature in signatures.items():
        # Create dataframe
        enrichment_results = results['pathway_enrichment'][label]
        enrichment_dataframe = pd.concat([enrichment_results['upregulated'], enrichment_results['downregulated']])

        # Plot barcharts
        libraries = enrichment_dataframe['gene_set_library'].unique()   
        if (len(libraries)):
            display(Markdown("## Pathway Enrichment Analysis"))
            display(Markdown("Biological pathways are sequences of interactions between biochemical compounds which play a key role in determining cellular behavior. Databases such as KEGG (Kanehisa et al. 2000), Reactome (Croft et al. 2014) and WikiPathways (Kelder et al. 2012) contain a large number of associations between such pathways and genes. This information can be leveraged by Enrichr to identify the biological pathways which are over-represented in the up-regulated and down-regulated genes identified by comparing two groups of samples."))
        
            for gene_set_library in libraries:
                # Display results
                plot_library_barchart(enrichment_results, gene_set_library, enrichment_results['signature_label'], enrichment_results['sort_results_by'], nr_genesets=nr_genesets, plot_type=plot_type)
        
            figure_counter = display_object(figure_counter, "Enrichment Analysis Results for {} in KEGG Pathways, WikiPathways and Reactome Pathways. The figure contains interactive bar charts displaying the results of the Gene Ontology enrichment analysis generated using Enrichr. The x axis indicates the -log10(P-value) for each term. Significant terms are highlighted in bold. Additional information about enrichment results is available by hovering over each bar.".format(label), istable=False)

In [ ]:
# Transcription Factor Enrichment Analysis
if "Transcription Factor" in enrichr_libraries and enrichr_success:
    display(Markdown("## Transcription Factor Enrichment Analysis"))
    display(Markdown("Transcription Factors (TFs) are proteins involved in the transcriptional regulation of gene expression. Databases such as ChEA (Lachmann et al. 2010) and ENCODE (Consortium, 2014) contain a large number of associations between TFs and their transcriptional targets. This information can be leveraged by Enrichr to identify the transcription factors whose targets are over-represented in the up-regulated and down-regulated genes identified by comparing two groups of samples."))

    # Initialize results
    results['tf_enrichment'] = {}

    # Loop through results
    for label, enrichr_results in results['enrichr'].items():
    #     # Run analysis
        results['tf_enrichment'][label] = get_enrichr_result_tables_by_library(enrichr_results=enrichr_results, signature_label=label)
        table_counter = display_table(results['tf_enrichment'][label], "Transcription Factor", table_counter)


In [ ]:
# Kinase Enrichment Analysis
if "Kinase" in enrichr_libraries and enrichr_success:
    display(Markdown("## Kinase Enrichment Analysis"))
    display(Markdown("Protein kinases are enzymes that modify other proteins by chemically adding phosphate groups. Databases such as KEA (Lachmann et al. 2009) contain a large number of associations between kinases and their substrates. This information can be leveraged by Enrichr to identify the protein kinases whose substrates are over-represented in the up-regulated and down-regulated genes identified by comparing two groups of samples."))

    # Initialize results
    results['kinase_enrichment'] = {}

    # Loop through results
    for label, enrichr_results in results['enrichr'].items():
        # Run analysis
        results['kinase_enrichment'][label] = get_enrichr_result_tables_by_library(enrichr_results=enrichr_results, signature_label=label, library_type="ke")

        # Display results
        table_counter = display_table(results['kinase_enrichment'][label], "Kinase", table_counter)

In [ ]:
# miRNA Enrichment Analysis
if "miRNA" in enrichr_libraries and enrichr_success:
    display(Markdown("## miRNA Enrichment Analysis"))
    display(Markdown("microRNAs (miRNAs) are small non-coding RNA molecules which play a key role in the post-transcriptional regulation of gene expression. Databases such as TargetScan (Agarwal et al. 2015) and MiRTarBase (Chou et al. 2016) contain a large number of associations between miRNAs and their targets. This information can be leveraged by Enrichr to identify the miRNAs whose targets are over-represented in the up-regulated and down-regulated genes identified by comparing two groups of samples."))

    results['mirna_enrichment'] = {}

    # Loop through results
    for label, enrichr_results in results['enrichr'].items():
        # Run analysis
        results['mirna_enrichment'][label] = get_enrichr_result_tables_by_library(enrichr_results=enrichr_results, signature_label=label, library_type="mirna")

        # Display results
        table_counter = display_table(results['mirna_enrichment'][label], "miRNA", table_counter)


# LINCS L1000 Small Molecule Candidates

In [ ]:
%%appyter markdown
{% if small_molecule_method.value == "L1000CDS2" %}
# L1000CDS2 Query
L1000CDS2 (Duan et al. 2016) is a web-based tool for querying gene expression signatures against signatures created from human cell lines treated with over 20,000 small molecules and drugs for the LINCS project. It is commonly used to identify small molecules which mimic or reverse the effects of a gene expression signature generated from a differential gene expression analysis.
{% endif %}

In [ ]:
%%appyter code_exec
{% if small_molecule_method.value == "L1000CDS2" %}
# Initialize results
results['l1000cds2'] = {}

try:
    # Loop through signatures
    for label, signature in signatures.items(): 
        # Run analysis
        results['l1000cds2'][label] = run_l1000cds2(signature=signature, nr_genes=l1000_topk, signature_label=label, plot_type=plot_type)

        # Display results
        figure_counter = plot_l1000cds2(results['l1000cds2'][label], counter=figure_counter, nr_drugs=nr_drugs)
except Exception as e:
    print(f"L1000CDS2 Error! '{e}'")
    print ('Skip analysis')

{% endif %}

In [ ]:
%%appyter markdown
{% if small_molecule_method.value == "L1000FWD" %}
# L1000FWD Query
L1000FWD (Wang et al. 2018) is a web-based tool for querying gene expression signatures against signatures created from human cell lines treated with over 20,000 small molecules and drugs for the LINCS project.
{% endif %}

In [ ]:
%%appyter code_exec
{% if small_molecule_method.value == "L1000FWD" %}
# Initialize results
results['l1000fwd'] = {}

# Loop through signatures
try:
    for label, signature in signatures.items():
        display(Markdown("*L1000FWD for {}*".format(label)))
    
        # Run analysis
        results['l1000fwd'][label] = run_l1000fwd(signature=signature, signature_label=label, nr_genes=l1000_topk)

        # Display results
        table_counter = plot_l1000fwd(results['l1000fwd'][label], counter=table_counter)
except Exception as e:
    print(f"L1000FWD Error! '{e}'")
    print ('Skip analysis')

{% endif %}

# References

Agarwal, Vikram, et al. "Predicting effective microRNA target sites in mammalian mRNAs." elife 4 (2015): e05005.
<br>
Ashburner, M., Ball, C.A., Blake, J.A., Botstein, D., Butler, H., Cherry, J.M., Davis, A.P., Dolinski, K., Dwight, S.S. and Eppig, J.T. (2000) Gene Ontology: tool for the unification of biology. Nature genetics, 25, 25.
<br>
Chou, Chih-Hung, et al. "miRTarBase 2016: updates to the experimentally validated miRNA-target interactions database." Nucleic acids research 44.D1 (2016): D239-D247.
<br>
Clark, N.R. and Ma’ayan, A. (2011) Introduction to statistical methods to analyze large data sets: principal components analysis. Sci. Signal., 4, tr3-tr3.
<br>
Clark, Neil R., et al. "The characteristic direction: a geometrical approach to identify differentially expressed genes." BMC bioinformatics 15.1 (2014): 79.
<br>
Consortium, E.P. (2004) The ENCODE (ENCyclopedia of DNA elements) project. Science, 306, 636-640.
<br>
Croft, David, et al. "The Reactome pathway knowledgebase." Nucleic acids research 42.D1 (2014): D472-D477.
<br>
Duan, Q., et al. "L1000cds2: Lincs l1000 characteristic direction signatures search engine. NPJ Syst Biol Appl. 2016; 2: 16015." (2016).
<br>
Fernandez, Nicolas F., et al. "Clustergrammer, a web-based heatmap visualization and analysis tool for high-dimensional biological data." Scientific data 4 (2017): 170151.
<br>
Kanehisa, M. and Goto, S. (2000) KEGG: kyoto encyclopedia of genes and genomes. Nucleic acids research, 28, 27-30.
<br>
Kelder, Thomas, et al. "WikiPathways: building research communities on biological pathways." Nucleic acids research 40.D1 (2012): D1301-D1307.
<br>
Kuleshov, M.V., Jones, M.R., Rouillard, A.D., Fernandez, N.F., Duan, Q., Wang, Z., Koplev, S., Jenkins, S.L., Jagodnik, K.M. and Lachmann, A. (2016) Enrichr: a comprehensive gene set enrichment analysis web server 2016 update. Nucleic acids research, 44, W90-W97.
<br>
Lachmann, A., Xu, H., Krishnan, J., Berger, S.I., Mazloom, A.R. and Ma'ayan, A. (2010) ChEA: transcription factor regulation inferred from integrating genome-wide ChIP-X experiments. Bioinformatics, 26, 2438-2444.
<br>
Lachmann, Alexander, and Avi Ma'ayan. "KEA: kinase enrichment analysis." Bioinformatics 25.5 (2009): 684-686.
<br>
Ritchie, Matthew E., et al. "limma powers differential expression analyses for RNA-sequencing and microarray studies." Nucleic acids research 43.7 (2015): e47-e47.
<br>
Wang, Zichen, et al. "L1000FWD: fireworks visualization of drug-induced transcriptomic signatures." Bioinformatics 34.12 (2018): 2150-2152.